# Gen Code 43
GenCode 43 released 08.02.2023 (Feb 8)  
My download started Feb 27.   

Get FASTA and GFF3 from here: https://www.gencodegenes.org/human/  
Scroll down to Fasta files   
Click on Protein-coding transcript sequences = gencode.v43.pc_transcripts.fa   
Click on Long non-coding RNA transcript sequences = gencode.v43.lncRNA_transcripts.fa   
Scroll down to GFF3 files    
Click on Basic gene annotation on chromosomes (the main file for most uers) = gencode.v43.basic.annotation.gff3

GenCode [biotypes](https://www.gencodegenes.org/pages/biotypes.html)
describes for example 'lncRNA' 

GenCode [tags](https://www.gencodegenes.org/pages/tags.html)
describes for example 'appris_principal_1' Where the transcript expected to code for the main functional isoform...

## Decide to use Basic Annotation
Using v42 in Sep 2022, 
we previously used the full "annotation" file which has 252416 transcripts. 
Our filters reduced this to 114760 (29K non-coding and 86L coding).
The "basic.annotation" file had 117681 transcripts.
I expect "basic" probably removes the same ones we filtered. 

It is unclear exactly what "basic" means for lncRNA. According to the FAQ...
The transcripts tagged as "basic" form part of a subset of representative transcripts for each gene. This subset prioritises full-length protein coding transcripts over partial or non-protein coding transcripts within the same gene, and intends to highlight those transcripts that will be useful to the majority of users.

In [1]:
from datetime import datetime
print(datetime.now())

2023-02-27 18:21:20.773785


In [2]:
# GenCode inputs
DATA_DIR = '/Users/jasonmiller/WVU/Localization/GenCode/GenCode43/'
ANNOTATION = 'gencode.v43.basic.annotation.gff3'
NONCODING_SEQUENCE = 'gencode.v43.lncRNA_transcripts.fa'   
CODING_SEQUENCE = 'gencode.v43.pc_transcripts.fa'    
# GenCode outputs
CODING_CSV_ALL = 'gencode_v43.all_pc_transcripts.csv'        
NONCODING_CSV_ALL = 'gencode_v43.all_lncRNA_transcripts.csv'   
CODING_CSV_CANON = 'gencode_v43.canon_pc_transcripts.csv'        
NONCODING_CSV_CANON = 'gencode_v43.canon_lncRNA_transcripts.csv'   
# Atlas inputs
ATLAS_DIR = '/Users/jasonmiller/WVU/Localization/LncAtlas/'
ATLAS_FILE = 'lncATLAS_all_data_RCI.csv'

## lncAtlas Gene IDs
Load the list of distinct genes in lncAtlas.
Only examine lines where field 2 is CNRCI and field 3 is not NA.
Load the ESNG IDs.
Remove duplicate IDs.

In [12]:
def load_atlas_genes(filepath):
    '''Expect a CSV file.'''
    pc_genes = set()
    nc_genes = set()
    with open (filepath,'r') as handle:
        header = None
        for row in handle:
            if header is None:
                header = row
            else:
                fields = row.split(',')
                gene_id = fields[0]  # like ENSG00000000003
                rci_type = fields[2]   
                value = fields[3]
                gene_type = fields[5] # field 5 same as field 6 in all cases
                if (value != 'NA' and rci_type == 'CNRCI'):
                    if gene_type == 'coding':
                        pc_genes.add(gene_id)  # set removes dupes
                    elif gene_type == 'nc':
                        nc_genes.add(gene_id)
    return pc_genes, nc_genes

In [13]:
print(datetime.now())
atlas_pc_genes,atlas_nc_genes = load_atlas_genes(ATLAS_DIR+ATLAS_FILE)
print('Atlas pc genes:', len(atlas_pc_genes))
print('Atlas nc genes:', len(atlas_nc_genes))

2023-02-27 18:25:21.918262
Atlas pc genes: 17770
Atlas nc genes: 6768


## GenCode annotation

In [14]:
def load_annotated_transcripts(filepath,all_transcripts=True):
    '''Expect a GFF3 file.'''
    pc_tids = set()
    nc_tids = set()
    with open (filepath,'r') as handle:
        for row in handle:
            columns = row.split('\t')
            # Avoid comment lines and read only transcript lines.
            if len(columns)>=9 and columns[2] == 'transcript':
                # The data we need is in column 9, the so-called comments,
                # listed as name=value pairs separated by semicolon.
                comments = columns[8] 
                pairs = comments.split(';') 
                tid = None
                gtype = None
                ttype = None
                canonical = False
                for pair in pairs:
                    # Look for certain tags and retain their values.
                    if pair.startswith('ID=ENST'):
                        tid = pair[3:] 
                    elif pair.startswith('gene_type='):
                        gtype = pair[10:] 
                    elif pair.startswith('transcript_type='):
                        ttype = pair[16:]
                    elif not all_transcripts and pair.startswith('tag='):
                        if 'Ensembl_canonical' in pair:
                            canonical = True
                if ttype is not None:
                    if tid is None:
                        raise Exception('transcript type without ID')
                    if ttype==gtype: 
                        if all_transcripts or canonical is True:
                            if ttype=='protein_coding':
                                pc_tids.add(tid)
                            elif ttype=='lncRNA':
                                nc_tids.add(tid)
    return pc_tids, nc_tids

In [15]:
print(datetime.now())
gencode_pc_transcripts,gencode_nc_transcripts = \
    load_annotated_transcripts(DATA_DIR+ANNOTATION,True)
print('Gencode all pc transcripts:', len(gencode_pc_transcripts))
print('Gencode all nc transcripts:', len(gencode_nc_transcripts))
canonical_pc_transcripts,canonical_nc_transcripts = \
    load_annotated_transcripts(DATA_DIR+ANNOTATION,False)
print('Gencode canonical pc transcripts:', len(canonical_pc_transcripts))
print('Gencode canonical nc transcripts:', len(canonical_nc_transcripts))

2023-02-27 18:25:34.933040
Gencode all pc transcripts: 63752
Gencode all nc transcripts: 29147
Gencode canonical pc transcripts: 19688
Gencode canonical nc transcripts: 17153


## Make sequence files
Parse FASTA, extract sequences of interest, write CSV.
The rest of our pipeline uses CSV because it is easier to process than FASTA.

In [16]:
class fasta_reader():
    '''Expect human transcripts FASTA file from GenCode.'''
    def __init__(self,infile,outfile,biotype):
        '''Biotype should be either 'pc' or 'lncRNA'.'''
        self.infile = infile
        self.outfile = outfile
        self.biotype = biotype
        self.FASTA_DEFCHAR = '>'  # signals a defline = definition line
        self.allow_genes = None
        self.allow_transcripts = None
        self.transcripts_per_gene = None
        self.headers='transcript_id,gene_id,biotype,length,sequence\n'
    def get_transcripts_per_gene(self):
        return self.transcripts_per_gene
    def allow_these_genes(self,genes:set):
        self.allow_genes = genes
    def allow_these_transcripts(self,trans:set):
        self.allow_transcripts = trans
    def print_one_sequence(self,handle,tran,gene,seq):
        allow_genes = self.allow_genes
        allow_trans = self.allow_transcripts
        if seq is not None:
            # sequence is None when we encounter the first defline
            if allow_genes is None or gene in allow_genes:
                if allow_trans is None or tran in allow_trans:
                    biotype = self.biotype
                    length = str(len(seq))
                    outstr = ','.join((tran,gene,biotype,length,seq))
                    handle.write(outstr+'\n')
                    self.count_out += 1
                    if gene not in self.transcripts_per_gene:
                        self.transcripts_per_gene[gene]=0
                    self.transcripts_per_gene[gene] += 1  
    def fasta_to_csv(self):
        self.count_in = 0
        self.count_out = 0
        self.transcripts_per_gene = {}
        with open(self.outfile,'w') as handle:
            handle.write(self.headers)
            with open(self.infile,'r') as fasta:
                transcript_id = None
                gene_id = None
                next_seq = None
                for line in fasta:
                    if line[0]==self.FASTA_DEFCHAR:
                        # Wrap up the previous sequence before moving on to the next.
                        self.print_one_sequence(handle,transcript_id,gene_id,next_seq)
                        # The defline starts with '>'
                        # The defline has fields separated by vertical bar
                        self.count_in += 1
                        tokens = line.split('|')
                        transcript_id = tokens[0][1:] # chop off '>'
                        gene_id = tokens[1]   
                        version_index=gene_id.find('.')   
                        if version_index>=0:
                            # chop off version number, like the 2 in ENSG00000198888.2
                            gene_id = gene_id[:version_index]  
                        next_seq = ""   # get ready for one to many sequence lines
                    else:
                        # In FASTA format, one sequence may continue to next line
                        next_seq = next_seq + line.strip()  
            # Reading FASTA, be sure to process the last sequence.
            self.print_one_sequence(handle,transcript_id,gene_id,next_seq)
        return self.count_in,self.count_out

In [17]:
print(datetime.now())
infile = DATA_DIR + CODING_SEQUENCE
outfile = DATA_DIR + CODING_CSV_ALL
converter = fasta_reader(infile,outfile,'pc')
converter.allow_these_genes(atlas_pc_genes)
converter.allow_these_transcripts(gencode_pc_transcripts)
count_in,count_out=converter.fasta_to_csv()
print("All mRNA in GenCode and Atlas")
print(" Input sequences: %d" % count_in)
print("Output sequences: %d" % count_out)
print("Genes in Atlas: %d" % len(atlas_pc_genes))
print("Genes represented in output: %d" % len(converter.get_transcripts_per_gene()))

2023-02-27 18:26:02.821587
All mRNA in GenCode and Atlas
 Input sequences: 111276
Output sequences: 60136
Genes in Atlas: 17770
Genes represented in output: 17399


In [18]:
print(datetime.now())
infile = DATA_DIR + NONCODING_SEQUENCE
outfile = DATA_DIR + NONCODING_CSV_ALL
converter = fasta_reader(infile,outfile,'lncRNA')
converter.allow_these_genes(atlas_nc_genes)
converter.allow_these_transcripts(gencode_nc_transcripts)
count_in,count_out=converter.fasta_to_csv()
print("All lncRNA in GenCode and Atlas")
print(" Input sequences: %d" % count_in)
print("Output sequences: %d" % count_out)
print("Genes in Atlas: %d" % len(atlas_nc_genes))
print("Genes represented in output: %d" % len(converter.get_transcripts_per_gene()))

2023-02-27 18:26:28.270087
All lncRNA in GenCode and Atlas
 Input sequences: 58023
Output sequences: 11656
Genes in Atlas: 6768
Genes represented in output: 5767


In [19]:
print(datetime.now())
infile = DATA_DIR + CODING_SEQUENCE
outfile = DATA_DIR + CODING_CSV_CANON
converter = fasta_reader(infile,outfile,'pc')
converter.allow_these_genes(atlas_pc_genes)
converter.allow_these_transcripts(canonical_pc_transcripts)
count_in,count_out=converter.fasta_to_csv()
print("Canonical mRNA in GenCode and Atlas")
print(" Input sequences: %d" % count_in)
print("Output sequences: %d" % count_out)
print("Genes in Atlas: %d" % len(atlas_pc_genes))
print("Genes represented in output: %d" % len(converter.get_transcripts_per_gene()))

2023-02-27 18:27:02.707729
Canonical mRNA in GenCode and Atlas
 Input sequences: 111276
Output sequences: 17415
Genes in Atlas: 17770
Genes represented in output: 17398


In [20]:
print(datetime.now())
infile = DATA_DIR + NONCODING_SEQUENCE
outfile = DATA_DIR + NONCODING_CSV_CANON
converter = fasta_reader(infile,outfile,'lncRNA')
converter.allow_these_genes(atlas_nc_genes)
converter.allow_these_transcripts(canonical_nc_transcripts)
count_in,count_out=converter.fasta_to_csv()
print("Canonical lncRNA in GenCode and Atlas")
print(" Input sequences: %d" % count_in)
print("Output sequences: %d" % count_out)
print("Genes in Atlas: %d" % len(atlas_nc_genes))
print("Genes represented in output: %d" % len(converter.get_transcripts_per_gene()))

2023-02-27 18:27:05.933339
Canonical lncRNA in GenCode and Atlas
 Input sequences: 58023
Output sequences: 5042
Genes in Atlas: 6768
Genes represented in output: 5033


In [21]:
tpg = converter.get_transcripts_per_gene()
for gene in atlas_nc_genes:
    if gene not in tpg.keys():
        print(gene)

ENSG00000274628
ENSG00000259146
ENSG00000237764
ENSG00000272551
ENSG00000235597
ENSG00000279415
ENSG00000196951
ENSG00000231064
ENSG00000273352
ENSG00000234352
ENSG00000279050
ENSG00000279118
ENSG00000242086
ENSG00000250903
ENSG00000279908
ENSG00000236924
ENSG00000273036
ENSG00000239332
ENSG00000279700
ENSG00000175699
ENSG00000279409
ENSG00000214725
ENSG00000279759
ENSG00000276995
ENSG00000226472
ENSG00000249346
ENSG00000235092
ENSG00000250159
ENSG00000222020
ENSG00000240687
ENSG00000237877
ENSG00000279656
ENSG00000232698
ENSG00000280406
ENSG00000278983
ENSG00000236404
ENSG00000265787
ENSG00000262879
ENSG00000234290
ENSG00000256164
ENSG00000279748
ENSG00000279592
ENSG00000262165
ENSG00000282851
ENSG00000237647
ENSG00000182310
ENSG00000279315
ENSG00000224745
ENSG00000262668
ENSG00000267053
ENSG00000255345
ENSG00000280187
ENSG00000223685
ENSG00000282907
ENSG00000278974
ENSG00000282906
ENSG00000258702
ENSG00000280707
ENSG00000279312
ENSG00000267659
ENSG00000253524
ENSG00000205622
ENSG0000

print('ENSG00000274628')
This nc gene is in lncAtlas with a CNRCI.
This nc gene is in LNCipedia.

The gene is in the GenCode GFF3 file
with an Ensembl_canonical transcript(ENST00000659401.1).
The gene is marked gene_type=transcribed_unprocessed_pseudogene,
which according to GenCode tags, means 
"...presence of locus-specific transcripts indicates expression."
This gene and transcript is missing from the GenCode FASTA file.
Ensembl says...
This gene was missing from GRCh37 but found on chr9 of GRCh38.
No RefSeq equivalent.
